In [ ]:
#!pip install torchvision

In [ ]:
import Config.ConfigMain as conf
import Config.ConfigPaths as paths
from Config.Location import location
from Secrets import *

from __future__ import print_function, division

import torch
import pandas as pd
import torch.nn as nn

import torch.optim as optim

from torch.optim import lr_scheduler

import numpy as np


import torchvision
from torchvision import datasets, models, transforms

import matplotlib.pyplot as plt

import time

import os

import copy
import pandas as pd

plt.ion() #interactive mode


In [ ]:
#os.listdir('./train_set/train_set')

In [5]:
#resize, pixels. resize depending on data, we have to explore other sizes to check for performance
training_transforms=transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor()])
print(training_transforms)

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
    ToTensor()
)


In [6]:
train_dataset=torchvision.datasets.ImageFolder(root=training_data,transform=training_transforms)


FileNotFoundError: [Errno 2] No such file or directory: './train_set'

In [7]:
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=32,shuffle=False)

In [8]:

# def get_mean_and_std(loader):
#     mean=0.
#     std=0.
#     total_images_count=0
#     for images,_ in loader:
#         image_count_in_a_batch=images.size(0)
#         images=images.view(image_count_in_a_batch, images.size(1), -1)
#         mean+=images.mean(2).sum(0)
#         std+=images.std(2).sum(0)
#         total_images_count+=image_count_in_a_batch
#     mean/=total_images_count
#     std/=total_images_count
#     return mean,std
    

In [9]:
# get_mean_and_std(train_loader)

In [10]:
train_dataset_path='./train_set'
test_dataset_path= './test_set'

In [11]:
#image data preparation 
#normalized data=> image=(image-mean)/std

mean=[0.6366, 0.5437, 0.4454]
std=[0.2235, 0.2422, 0.2654]
train_transforms=transforms.Compose([

                                 transforms.Resize((224,224)),

                                 transforms.RandomHorizontalFlip(),
    
                                 transforms.RandomRotation(10),

                                 transforms.ToTensor(),

                                 transforms.Normalize(torch.Tensor(mean),torch.Tensor(std))
])

test_transforms=transforms.Compose([
                                 transforms.Resize((224,224)),
    
                                 transforms.ToTensor(),

                                 transforms.Normalize(torch.Tensor(mean),torch.Tensor(std))
])

In [12]:
train_dataset=torchvision.datasets.ImageFolder(root=train_dataset_path,transform=train_transforms)
test_dataset=torchvision.datasets.ImageFolder(root=test_dataset_path,transform=train_transforms)

In [13]:
#print(test_dataset)

In [14]:
# def show_transf_images(data):
#     loader=torch.utils.data.DataLoader(data,batch_size=6, shuffle=True)
#     batch=next(iter(loader))
#     images,labels=batch
    
#     grid=torchvision.utils.make_grid(images,nrow=3)
#     plt.figure(figsize=(11,11))
#     plt.imshow(np.transpose(grid,(1,2,0)))
#     print('labels:',labels)
    

In [15]:
# show_transf_images(test_dataset)



In [18]:
#Mini-Batch Gradient Descent, start with 32 and explore to increase performance
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=32,shuffle=True)
                           